In [2]:
# Pour gerer et manipuler les structures de données multidimentionnels
import pandas as pd  

# Pour gerer les calculs matriciels
import numpy as np  

# Pour visualiser les données
import matplotlib.pyplot as plt 
from fanalysis.mca import MCA

#mt est une instance de la classe math
import math as mt 


In [ ]:
data = pd.read_excel(r'C:/Users/DELL/Documents/Data_Methodes_Factorielles.xlsx',sheet_name="ACM_CANINES",index_col=0)
print(data)

In [ ]:
DActives = [x for x in data.columns if data[x].dtype == 'object']
# On récupere les colonnes du dataSet.
variable = data[DActives]
print(variable)
# Le nombre des individus
n = variable.shape[0]
# Le nombre des variables
p = variable.shape[1]
print("le nombre des individus :",n)
print("le nombre des variables :",p)
# pd.get_dummies() est utilisé pour la manipulation des données. Il convertit les données catégorielles en variables 
# fictives ou indicatrices.  
# Construire le tableau disjonctif complet
TDC = pd.get_dummies(variable,prefix='',prefix_sep='')
print(TDC)
 # Le nombre total de modalité
M = TDC.shape[1]
print("Le nombre total de modalité",M)
#instanciation de la méthode mca qu'est disponible dans fanalysis
acm = MCA(row_labels=TDC.index,var_labels=TDC.columns) 
x=acm.fit(TDC.values)  
# nombre max de facteur (pour savoir la dimension de nouvelle espace)
#le nombre de facteurs extraits ne doit pas dépasser le nombre d'individus moins le nombre de variables. Cela garantit que vous ne créez pas plus de facteurs que le nombre d'observations que vous avez dans votre jeu de données
Hmax = M - p  
print("nombre max de facteur",Hmax)
# Détérmination des valeurs propres 
valeur_propre = pd.DataFrame(np.transpose(acm.eig_),columns = ['valeur propre','%','cumul %'])
print("les val propres :")
print(valeur_propre)

In [ ]:
# Eboulis des valeurs propres pour savoir les valeurs propres qui porte plus informations
#Son rôle est de vous aider à déterminer combien de composantes principales (ou facteurs) vous devez retenir pour réduire la dimensionnalité de vos données.
fix , ax = plt.subplots(figsize=(5,5)) 
ax.plot(range(0,len(valeur_propre)),acm.eig_[0],".-")  
ax.set_xlabel("nombre de facteurs")
ax.set_ylabel("valeur propre")
plt.title("Eboulis des valeus propres")
#seuil - Règle de kaiser
print(p)
ax.plot([1,Hmax],[1/p,1/p],"r-",linewidth=1) 
plt.show()

# Séléction les valeurs propres qui porte plus informations
valeur_qui_porte_plus_information = [acm.eig_[0][0] , acm.eig_[0][1]]
print("les valeurs propres qui portent plus d'information")
print(valeur_qui_porte_plus_information)
 # Affichage les coordonnes des facteurs.
data_facteur = pd.DataFrame(acm.col_coord_[0:len(TDC.columns),:2],columns=['F1','F2'],index=TDC.columns) 
data_facteurrow=pd.DataFrame(acm.row_coord_[:,:2],index=TDC.index,columns=['Coord.F1','Coord.F2'])
print(data_facteur)
print(data_facteurrow)
    # Pour détermine l'échelle correspondant 
maxii_F1 = max(list(data_facteur['F1'])) #pour recuperer la valeur maximale dans la 1ere facteur F1
minii_F1 = min(list(data_facteur['F1']))  #pour recuperer la valeur minimale dans la 1ere facteur F1
maxii_F2 = max(list(data_facteur['F2']))  #pour recuperer la valeur maximale dans le 2eme facteur F2
minii_F2 = min(list(data_facteur['F2']))  #pour recuperer la valeur minimale dans le 2eme facteur F2

if (maxii_F1>0):
        maxi_F1 = mt.floor(maxii_F1)+1
else:
        maxi_F1 = mt.floor(maxii_F1)

if (minii_F1<0): 
        mini_F1 = mt.floor(minii_F1)
else:
        mini_F1 =mt.floor(minii_F1)-1

if (maxii_F2>0): 
        maxi_F2 = mt.floor(maxii_F2)+1
else:
        maxi_F2 = mt.floor(maxii_F2)

if (minii_F2<0): 
        mini_F2 = mt.floor(minii_F2)
else:
        mini_F2 =mt.floor(minii_F2)-1
        
        


In [ ]:
# Projection des modalités sur le nouveau plan
fix , ax = plt.subplots(figsize=(7,7))  
ax.axis([mini_F1,maxi_F1,mini_F2,maxi_F2])
ax.plot([mini_F1,maxi_F1],[0,0],color='red',linestyle='--')
ax.plot([0,0],[mini_F2,maxi_F2],color='red',linestyle='--')
ax.set_xlabel("Dim.1 ")
ax.set_ylabel("Dim.2")
plt.title("Représentation des points-modalité")
for i in range(TDC.shape[1]):
        ax.text(acm.col_coord_[i,0],acm.col_coord_[i,1],TDC.columns[i],color='black')  
plt.show()


In [ ]:
# Projection des individus 
fix , ax = plt.subplots(figsize=(7,7))  
ax.axis([mini_F1,maxi_F1,mini_F2,maxi_F2])   
ax.plot([mini_F1,maxi_F1],[0,0],color='silver',linestyle='--')
ax.plot([0,0],[mini_F2,maxi_F2],color='silver',linestyle='--')
ax.set_xlabel("Dim1")
ax.set_ylabel("Dim2")
plt.title("Representation des individus")
for i in range(TDC.shape[0]):
    ax.text(acm.row_coord_[i,0],acm.row_coord_[i,1],TDC.index[i],color='red')   
plt.show()
    
   

In [ ]:
 
# Projection les individus et les modalité 
fix , ax = plt.subplots(figsize=(10,10))  #pour traver le cadre de representation
ax.plot([mini_F1,maxi_F1],[0,0],color='silver',linestyle='--')
ax.plot([0,0],[mini_F2,maxi_F2],color='silver',linestyle='--')
ax.set_xlabel("Dim1")
ax.set_ylabel("Dim2")
plt.title("Representation des individus et des modalité")
for i in range(TDC.shape[1]):
        ax.text(acm.col_coord_[i,0],acm.col_coord_[i,1],TDC.columns[i],color='black') 
for j in range(TDC.shape[0]): 
        ax.text(acm.row_coord_[j,0],acm.row_coord_[j,1],TDC.index[j],color='red') 
plt.show()